### FastAPI 설치

In [2]:
!pip install fastapi uvicorn nest-asyncio pyngrok

### FastAPI 서버 코드 작성

In [5]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [4]:
# Colab 환경에서의 이벤트 루프 수정
nest_asyncio.apply()

# FastAPI 앱 초기화
app = FastAPI()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
!ls "/drive/shared-with-me"

ls: cannot access '/drive/shared-with-me': No such file or directory


In [6]:
# 모델 및 토크나이저 로드 (Google Drive 경로에서)
model_dir = "/content/drive/MyDrive/t5_fine_tuned_keywords_sentiments"
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)

OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/t5_fine_tuned_keywords_sentiments'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
# 입력 데이터 형식 정의
class InputData(BaseModel):
    input_text: str

@app.post("/generate")
async def generate(input_data: InputData):
    # 입력 텍스트를 토큰화
    input_ids = tokenizer.encode(input_data.input_text, return_tensors="pt")

    # 모델로 예측 수행
    outputs = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)

    # 결과 디코딩
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {"generated_text": result}